In [1]:
import json
import openai
import os

from PIL import Image
import matplotlib.pyplot as plt

from rapidocr_onnxruntime import RapidOCR
# from rapidocr_paddle import RapidOCR

In [2]:
from docling.datamodel.pipeline_options import (
    AcceleratorDevice,
    AcceleratorOptions,
    RapidOcrOptions,
    OcrOptions
)

In [3]:
import base64

def encode_image(image_path):
	with open(image_path, "rb") as image_file:
		return base64.b64encode(image_file.read()).decode('utf-8')

In [5]:
# args
# https://github.com/RapidAI/RapidOCR/blob/fecad1f7882bd4bf6916f709dfdb8889c3c9e9af/python/rapidocr_paddle/utils/parse_parameters.py#L23
# RapidOCR
# https://rapidai.github.io/RapidOCRDocs/install_usage/api/RapidOCR/#__init__-source
# https://github.com/RapidAI/RapidOCR/blob/main/python/rapidocr_paddle/main.py
reader = RapidOCR(
    # no_rec = True,
    use_rec = True,
    rec_model_path="/Users/id4thomas/models/docling/paddle/korean_PP-OCRv4_rec_infer/model.onnx",
    rec_keys_path="/Users/id4thomas/models/docling/paddle/korean_PP-OCRv4_rec_infer/korean_dict.txt",
    
    
    # rec_model_path="/home/users/yrsong/research/2501-01-doc-parsing/models/mineru/pdf-extract-kit/models/OCR/PaddleOCR/rec/ch_PP-OCRv4_rec/inference.pdmodel"
    # text_score=0.4,
    # rec_model_path="/home/users/yrsong/research/2501-01-doc-parsing/models/docling/ch_PP-OCRv4_rec_server_infer.onnx",
    # rec_model_path="/home/users/yrsong/research/2501-01-doc-parsing/models/docling/rec/korean_PP-OCRv3_rec_infer",
    # rec_keys_path="/home/users/yrsong/research/2501-01-doc-parsing/models/docling/rec/korean_PP-OCRv3_rec_infer/korean_dict.txt",
    # rec_use_cuda=True
    # rec_use_cuda=False
)

In [6]:
img_path = 'results/docling/ai_parl_3/figure_export_without_table_structure/ai_parl_3-1.png'
result, elapse = reader(img_path)

In [7]:
result

[[[[711.0, 83.0], [1124.0, 85.0], [1124.0, 108.0], [711.0, 106.0]],
  '국회도서관|2023112812023-25호통권제237회',
  0.9693812211354573],
 [[[813.0, 130.0], [1124.0, 130.0], [1124.0, 170.0], [813.0, 170.0]],
  '최신외국입법정보',
  0.9986552894115448],
 [[[1032.0, 453.0], [1128.0, 453.0], [1128.0, 484.0], [1032.0, 484.0]],
  '콩음은',
  0.8674750725428263],
 [[[128.0, 604.0], [1067.0, 605.0], [1067.0, 633.0], [128.0, 632.0]],
  'IY h션 百ㄹn lo記 [Y弓 곤싱 [Y눌 百l_눕z百 룸표記 Io롱s 弓czoz',
  0.5870448113014193],
 [[[150.0, 647.0], [1066.0, 647.0], [1066.0, 676.0], [150.0, 676.0]],
  '인구중3%에해당하는약152만5천명이고독사위험군으로추정되고있다고독사의',
  0.9995844444713077],
 [[[149.0, 690.0], [1067.0, 690.0], [1067.0, 716.0], [149.0, 716.0]],
  '주요원인으로청장년층은경제적문제와정서적불안70대이상노인층은정서적불안과',
  0.99955903678327],
 [[[151.0, 732.0], [525.0, 732.0], [525.0, 758.0], [151.0, 758.0]],
  '건강문제가가장큰것으로나타났다',
  0.9998569250106811],
 [[[136.0, 816.0], [1065.0, 816.0], [1065.0, 842.0], [136.0, 842.0]],
  '우리나라는2021년4월에고독사예방및관리에관한법률]을제정하고이를근거로하여',
  0.9972032996324393

In [8]:
'''[
    [boundingbox coordinates, text, score]
]
'''
print(result[0])
print(result[0][0])
print(result[0][1])
print(result[0][2])

[[[711.0, 83.0], [1124.0, 85.0], [1124.0, 108.0], [711.0, 106.0]], '국회도서관|2023112812023-25호통권제237회', 0.9693812211354573]
[[711.0, 83.0], [1124.0, 85.0], [1124.0, 108.0], [711.0, 106.0]]
국회도서관|2023112812023-25호통권제237회
0.9693812211354573


In [9]:
## Load Image
image = Image.open(img_path)

bounding_box_points = result[3]
print(bounding_box_points)

x_coords = [point[0] for point in bounding_box_points]
y_coords = [point[1] for point in bounding_box_points]

min_x, max_x = int(min(x_coords)), int(max(x_coords))
min_y, max_y = int(min(y_coords)), int(max(y_coords))

cropped_image = image.crop((min_x, min_y, max_x, max_y))

plt.imshow(cropped_image)
plt.axis("off")  # Hide axis for better visualization
plt.show()

[[[128.0, 604.0], [1067.0, 605.0], [1067.0, 633.0], [128.0, 632.0]], 'IY h션 百ㄹn lo記 [Y弓 곤싱 [Y눌 百l_눕z百 룸표記 Io롱s 弓czoz', 0.5870448113014193]


TypeError: 'float' object is not subscriptable

# OpenAI Version

In [ ]:
from typing import Any, Dict

class OpenAIVLMOcrOptions(OcrOptions):
    openai_base_url: str
    openai_api_key: str
    model: str
    system_prompt: str
    user_prompt: str
    generation_configs: Dict[str, Any]

In [ ]:
from docling.models.base_ocr_model import BaseOcrModel

class OpenAIVLMOcrModel(BaseOcrModel):
    pass